# Optimization
This tutorial shows how to use Comets to perform an optimization.

## The Brewery model

Like in the previous tutorial we are working with the Brewery model. It is a drink consumption model including stock level management and customer satisfaction dynamics. The model input parameters that we will deal with are: 

- NBWaiters: the number of waiters in the bar
- RestockQty: a fix quantity of drinks added to the current stock of the bar once it reaches a threshold value.

The quantity of interest for this experiment is the Stock. More precisely, we want to maintain the final stock level to 40 drinks. We want to answer the following question: **How to maintain the final stock as close as possible to 40 drinks, with respect to the number of waiters and the restock quantity?**

## The approach
An optimization requires to specify precisely the decision variables and the objective function. The objective function is the quantity we want to minimize (or maximize) and the decision variables are the levers we can change to reach this objective.  
In this study the decision variables are NbWaiters, RestockQty and the objective function is the following:
$$
f(Stock) = (40 - Stock)^2
$$
The optimization is performed on an explicitly defined task with inputs and outputs. Here the inputs of the task are NbWaiters, RestockQty (the decision variables of the optimization) and the output is $ (40 - Stock)^2 $ (the objective function of the optimization). 

## 1. Define the task on which to experiment
Comets always works with the same method: it performs analyses on tasks. Before defining any experiment we need to define the task on which it will be performed. A task behaves as a function that takes as input a ParameterSet and outputs another ParameterSet. It has a method evaluate that is in charge of performing the evaluation and returning the outputs.
During the task evaluation, the model is run, but when creating the task we also allow for some additional pre-processing (i.e encoding) of the input parameters or post-processing of the outputs of the model.

In [1]:
# code to display the diagram below
from IPython.display import Image
Image(url= "Comets_ModelTask.png", width=1000, height=1600)

To run the analysis we first need to import comets

In [2]:
import comets

In order to import the Brewery model we then need to specify the path to the project containing it.

In [3]:
#This example assumes that this notebook is in the folder containing the Brewery project
#and that the model has been compiled with the python wrappers activated.

from pathlib import Path
cwd = Path().resolve()

### Import the simulator
We need to import the simulator in order to allow comets to interact with it. In this example, we work with the Brewery simulator, which we can instantiate using a CosmoInterface as follows:

In [4]:
simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
)

For more details on how to instantiate the Brewery model, refer to the tutorial "Running simulatons with Comets".

### Define the input parameters of the task and the encoding for working with the model
The task should take as input a ParameterSet, which is a set of parameters, stored as a dictionary containing names of parameters and their values. In our case the input parameters are the NBWaiters and the RestockQty, which would take the form: 
{'NBWaiters': NBWaiters, 'RestockQty': RestockQty} 



In order for the model to recognize input parameters, we need to encode them to a ParameterSet where the keys are the datapaths of the corresponding attributes in the model, which for NBWaiters and RestockQty corresponds to: 

- 'Model::{Entity}MyBar::@NbWaiters'
- 'Model::{Entity}MyBar::@RestockQty'

The encoding function takes the ParameterSet:

{'NBWaiters': NBWaiters, 'RestockQty': RestockQty} 

and returns the ParameterSet:

{'Model::{Entity}MyBar::@NbWaiters': NbWaiters, 'Model::{Entity}MyBar::@RestockQty': RestockQty}.

In [5]:
def encoder(parameters):
    return {'Model::{Entity}MyBar::@NbWaiters': parameters['NbWaiters'],
            'Model::{Entity}MyBar::@RestockQty': parameters['RestockQty']}

### Define the quantity of interest for the experiment, which is the output of the task
The quantity we are interested in is the the square difference between the aimed stock and the actual stock. This quantity is not computed directly in the model, but can be computed from the model output. It is defined as:
$$
\text{quantity of interest} = (40 - Stock)^2
$$
The Stock is available in the model via its datapath: 'Model::{Entity}MyBar::@Stock'

A quantity of interest can be computed from the output of the model by creating a function that takes as input the CosmoInterface and returns a ParameterSet with the values of the quantity of interest.

In [6]:
def QuantityOfInterest(simulator):
    outputs = simulator.get_outputs(['Model::{Entity}MyBar::@Stock'])
    stock = outputs['Model::{Entity}MyBar::@Stock']
    return {'ObjectiveFunction' : (stock-40)**2}

### Declaring the Task

Once the inputs and outputs have been defined, the final step is to declare the task with its CosmoInterface, encode and get_outcomes method. Evaluating the task corresponds to encoding the inputs, running a simulation and then computing the quantity of interest with the get_outcomes function.

In [7]:
optimtask = comets.ModelTask(simulator, get_outcomes = QuantityOfInterest, encode = encoder)

## 2. Defining the decision variables space
Once the task has been defined, we need to specify the decision variables space. This is done by declaring the decision variables in the format described below. The space is a list of dictionaries with each of the dictionaries containing the following information: 

- **name** (str): the name of the decision variable
- **type** (str): the data type of the decision variable (float or int)
- **bounds** (list): the bounds inside which the decision variable belongs
- **shape** (optional, integer): the length of the list when the decision variable is a list of variables.

The currently available types are: **"float"**, **"int"**.



In the Brewery model, NbWaiters and RestockQty are two integers. The former ranges from 1 to 12 and the latter from 1 to 50.

In [8]:
space = [
    {
        "name": "NbWaiters",
        "type": "int",
        "bounds": [1, 12],
    },
    {
        "name": "RestockQty",
        "type": "int",
        "bounds": [1, 50],
    },
]

## 3. Declaring the Experiment
An optimization is an experiment that can take the following arguments:

- **space**: the decision variables space
- **task**: the task on which to perform the optimization
- **algorithm**: the optimization algorithm used ('CMA', 'TwoPointsDE', 'NelderMead' ...)
- **stop_criteria**: dictionary containing the stop criteria of the experiment. In our case we use ‘max_evaluations’ which corresponds to the number of task evaluation to perform. Other criteria are available, more information on them can be found in Comets' documentation. 
- **n_jobs**: the number of tasks that are evaluated in parallel. Defaults to 1 (no parallelization)
- **maximize**: whether the optimization is seeking for a maximum (in opposition to a minimum). Defaults to False.

In [9]:
opt = comets.Optimization(
    space = space,
    task = optimtask,
    algorithm = 'CMA',
    stop_criteria = {'max_evaluations': 150},
    n_jobs = 8,
    maximize = False,
)

## 4. Running the experiment

In [10]:
opt.run()

## 5. Displaying the results of the Experiment
The results of the analysis are stored in the attribute 'results', it contains the optimal decision variables and the optimal value of the objective function.

In [11]:
opt.results

{'Optimal variables': {'NbWaiters': 7, 'RestockQty': 35},
 'Optimal values': {'ObjectiveFunction': 0}}